In [ ]:
from PIL import Image
import numpy as np
from photorealistic_smoothing import photorealistic_smoothing
from secondary_smoothing import secondary_smoothing
from networks import VGGEncoder, VGGDecoder

In [ ]:
import numpy as np
from PIL import Image
import torch.nn as nn
import torch
from wct import wct

In [ ]:
content = Image.open('images/Tuebingen_Neckarfront.jpg')
content

In [ ]:
style = Image.open('styles/Henri Edmond Cross - Antibes.jpg')
style

In [ ]:
def to_tensor(x):
    """
    Arguments:
        x: an instance of PIL image.
    Returns:
        a float tensor with shape [1, 3, h, w],
        it represents a RGB image with
        pixel values in [0, 1] range.
    """
    x = np.array(x)
    x = torch.FloatTensor(x)
    return x.permute(2, 0, 1).unsqueeze(0).div(255.0)

In [ ]:
class PhotoWCT(nn.Module):

    def __init__(self):
        super(PhotoWCT, self).__init__()

        self.encoder = VGGEncoder()
        self.decoders = nn.ModuleDict({f'{i}': VGGDecoder(i) for i in [1, 2, 3, 4]})

    def forward(self, content, style):
        """
        Arguments:
            content: a float tensor with shape [].
            style: a float tensor with shape [].
        """
        with torch.no_grad():

            style_features, _ = self.encoder(style)
            x = content

            for i in [1, 2, 3, 4]:
                features, pooling_indices = self.encoder(x, level=i)
                
                A, B = features[i], style_features[i]
                _, d, h, w = A.size()
                _, d, h2, w2 = B.size()
                f = wct(A.view(d, h * w), B.view(d, h2 * w2))
                f = f.view(1, d, h, w)
                x = self.decoders[f'{i}'](f, pooling_indices)

        return x

In [ ]:
t = PhotoWCT()

In [ ]:
t.encoder.load_state_dict(torch.load('models/encoder.pth'))

for i, m in t.decoders.items():
    m.load_state_dict(torch.load(f'models/decoder{i}.pth'))

In [ ]:
t = t.cuda()

In [ ]:
output = t(to_tensor(content).cuda(), to_tensor(style).cuda())

In [ ]:
output = output.cpu().clamp(0.0, 1.0)[0].permute(1, 2, 0).numpy()
output = (255*output).astype('uint8')

In [ ]:
Image.fromarray(output)

In [ ]:
X = np.array(content)
r = photorealistic_smoothing(X, output)

In [ ]:
r2 = secondary_smoothing(r3, X)

In [ ]:
Image.fromarray(r)

In [ ]:
Image.fromarray(r2)

In [ ]:
from cv2.ximgproc import guidedFilter


def guided_filter_smoothing(Y, content, radius=35, epsilon=1e-3):
    """
    Arguments:
        Y, content: numpy uint8 arrays with shape [h, w, 3].
        radius: an integer.
        epsilon: a float number.  #  or 1e-2
    Returns:
        a numpy uint8 array with shape [h, w, 3].
    """
    return guidedFilter(guide=content, src=Y, radius=radius, eps=epsilon)

In [ ]:
r3 = guided_filter_smoothing(output, X)

In [ ]:
Image.fromarray(r3)